<a href="https://colab.research.google.com/github/Fenix198811/Fenix198811/blob/main/SimpleCNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
from torchvision.utils import make_grid

import numpy as np
import pandas as pd
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
# Convert MNIST Imange Files into a Tensor of 4-Dimension(num of image, num of batch, Height, Width)
transform = transforms.ToTensor()

In [3]:
# Train Data
train_data = datasets.MNIST(root="/cnn_data", train=True, download=True, transform=transform)


Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 9912422/9912422 [00:01<00:00, 5231431.85it/s]


Extracting /cnn_data/MNIST/raw/train-images-idx3-ubyte.gz to /cnn_data/MNIST/raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 28881/28881 [00:00<00:00, 154604.07it/s]


Extracting /cnn_data/MNIST/raw/train-labels-idx1-ubyte.gz to /cnn_data/MNIST/raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 1648877/1648877 [00:01<00:00, 1449436.36it/s]


Extracting /cnn_data/MNIST/raw/t10k-images-idx3-ubyte.gz to /cnn_data/MNIST/raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 4542/4542 [00:00<00:00, 9088992.73it/s]

Extracting /cnn_data/MNIST/raw/t10k-labels-idx1-ubyte.gz to /cnn_data/MNIST/raw



In [4]:
# Test Data
test_data = datasets.MNIST(root="/cnn_data", train=False, download=True, transform=transform)

In [5]:
train_data

Dataset MNIST
    Number of datapoints: 60000
    Root location: /cnn_data
    Split: Train
    StandardTransform
Transform: ToTensor()

In [6]:
test_data

Dataset MNIST
    Number of datapoints: 10000
    Root location: /cnn_data
    Split: Test
    StandardTransform
Transform: ToTensor()

In [7]:
ls

sample_data/


In [8]:
ls /cnn_data

MNIST/


In [9]:
# Create a small batch of size for images   10
train_loader = DataLoader(train_data, batch_size=10, shuffle=True)
test_loader = DataLoader(test_data, batch_size=10, shuffle=False)

In [41]:
#Define Our CNN Model
# 2 CNN layers,
class CNN(nn.Module):
  def __init__(self):
    super().__init__()
    self.seq1 = nn.Sequential(
      nn.Conv2d(1, 6, 3, 1),
      nn.ReLU(),
      nn.MaxPool2d(kernel_size=2, stride=2)
    )
    self.seq2 = nn.Sequential(
      nn.Conv2d(6, 16, 3, 1),
      nn.ReLU(),
      nn.MaxPool2d(kernel_size=2, stride=2)
    )
    # Fully connected layer
    self.seq3 = nn.Sequential(
      nn.Linear(5*5*16, 120),
      nn.ReLU(),
      nn.Linear(120, 84),
      nn.ReLU(),
      nn.Linear(84, 10),
      nn.LogSoftmax(dim=1)
    )

    self.seq = nn.Sequential(
      self.seq1,
      self.seq2,
      nn.Flatten(),
      self.seq3
    )

  def forward(self, X):
    X = self.seq(X)
    return X

In [42]:
# Create an Instance of our model
torch.manual_seed(68)
model = CNN()
model

CNN(
  (seq1): Sequential(
    (0): Conv2d(1, 6, kernel_size=(3, 3), stride=(1, 1))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (seq2): Sequential(
    (0): Conv2d(6, 16, kernel_size=(3, 3), stride=(1, 1))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (seq3): Sequential(
    (0): Linear(in_features=400, out_features=120, bias=True)
    (1): ReLU()
    (2): Linear(in_features=120, out_features=84, bias=True)
    (3): ReLU()
    (4): Linear(in_features=84, out_features=10, bias=True)
    (5): LogSoftmax(dim=1)
  )
  (seq): Sequential(
    (0): Sequential(
      (0): Conv2d(1, 6, kernel_size=(3, 3), stride=(1, 1))
      (1): ReLU()
      (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    )
    (1): Sequential(
      (0): Conv2d(6, 16, kernel_size=(3, 3), stride=(1, 1))
      (1): ReLU()
      (2): MaxPool2d(kernel_size=2, stride=2, padd

In [43]:
# Loss Function Optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)


In [44]:
import time
start_time = time.time()

# Create Vars
epochs = 5
train_losses = []
test_losses = []
train_correct = []
test_correct = []

# For loop of Epochs
for i in range(epochs):
  train_corr = 0
  test_corr = 0

  # Train
  for b, (X_train, y_train) in enumerate(train_loader):
    b+=1
    y_pred = model(X_train) # get predicated values from the training set batch
    loss = criterion(y_pred, y_train)

    predicated = torch.max(y_pred.data, 1)[1] # add up the number of correct predictions,
    batch_corr = (predicated == y_train).sum() # how many we got correct from this batch
    train_corr += batch_corr

    # Update parameters
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    # print out some results
    if b%600 == 0:
      print(f"Epoch: {i}, Batch: {b}, Loss: {loss.item()}")

  train_losses.append(loss)
  train_correct.append(train_corr)

  # Test
  with torch.no_grad():
    for b, (X_test, y_test) in enumerate(test_loader):
      y_val = model(X_test)
      predicated = torch.max(y_val.data, 1)[1]
      test_corr += (predicated == y_test).sum()

  loss = criterion(y_val, y_test)
  test_losses.append(loss)
  test_correct.append(test_corr)

end_time = time.time()
total_time = end_time - start_time
print(f"Training took: {total_time/60} minutes")

Epoch: 0, Batch: 600, Loss: 0.17682988941669464
Epoch: 0, Batch: 1200, Loss: 0.18436872959136963
Epoch: 0, Batch: 1800, Loss: 0.3749023973941803
Epoch: 0, Batch: 2400, Loss: 0.5074084401130676
Epoch: 0, Batch: 3000, Loss: 0.032824285328388214
Epoch: 0, Batch: 3600, Loss: 0.22680477797985077
Epoch: 0, Batch: 4200, Loss: 0.020403796806931496
Epoch: 0, Batch: 4800, Loss: 0.006631325930356979
Epoch: 0, Batch: 5400, Loss: 0.0018949664663523436
Epoch: 0, Batch: 6000, Loss: 0.008881835266947746
Epoch: 1, Batch: 600, Loss: 0.005394080653786659
Epoch: 1, Batch: 1200, Loss: 0.0028350783977657557
Epoch: 1, Batch: 1800, Loss: 0.02368948422372341
Epoch: 1, Batch: 2400, Loss: 0.40532928705215454
Epoch: 1, Batch: 3000, Loss: 0.0012125405482947826
Epoch: 1, Batch: 3600, Loss: 0.01690104976296425
Epoch: 1, Batch: 4200, Loss: 0.016639549285173416
Epoch: 1, Batch: 4800, Loss: 0.016239095479249954
Epoch: 1, Batch: 5400, Loss: 0.0005746050737798214
Epoch: 1, Batch: 6000, Loss: 0.006660012062638998
Epoch: 2